# Question 2: Warming up with Quadratic Equations

## 2(a) Experiment with syntax variations

In [11]:
function real_roots_of_quadratic(a::Number, b::Number, c::Number)
    a == 0 && throw(DomainError(a, "argument must be nonzero"))

    roots::Array{Float64} = []
    Δ = b^2 - 4a*c

    if Δ < 0 
        roots = []
    elseif Δ == 0
        roots = [-b/(2a)]
    else
        roots = (-b .+ [√Δ, -√Δ]) / (2a)
    end
    return roots 
end

real_roots_of_quadratic(-1,5,-6)

2-element Vector{Float64}:
 2.0
 3.0

1. The code would still work because typing is not mandatory, however, my linter and IDE analysis tools might be a little messed up now, because the function can recieve any data as arguments instead of the numbers it expects. This could lead to unexpected or strange behaviours (like, if a, b, and c were string how would the delta varaible be caluclated? how would the roots be calculated?)

2. 4ac doesn't work! Variable names can't start with a 4, so Julia is smart enough to know that 4a means 4 times some variable called a. However, 4ac is arbitrary: do i want 4 times some varaible called a times some variable called c, or do i want one multiplication of 4 and some variable called "ac." When it runs, we get "ac is not defined" (ACTUALLY, 2 letter names are allowed, i tested this by starting the function with ac=1, and it ran just fine! The problem is it knows i want 4 times some varaible, and if ac is defined, then itll use ac. howver, we run into problems when "a" "c" and "ac" are all defined, however this issue is not brought up by the complier or the linter, so its up to the programmer to be smart about their naming conventions!)

3. a syntax error, specifically (ParseError: unexpected '=')

4. Yes, .+ [Del, -Del] worked. No, + [Del, -Del] did not work, i got a MethodError: no method matching +(::Int64, ::Vector{Float64}). It also gave me an error hint saying to use .+ for element-wise addition.

5. Yes, it works fine. Julia returns the result of the most recent command by default, so return is not strictly needed.

6. `real_roots_of_quadratic(0,5,-6)` returns `2-element Vector{Float64}: NaN -Inf`. This is because b^2 - 4ac = b^2 > 0 ensures we divide by 0 when we calculate the roots (.../2a=.../2(0)=.../0).

7. Yes, this seems to work, in the sense that my code doesn't crash out anymore

8. Yes this works. When Julia assess/executes the Boolean statement `a == 0 && return "Error - ..."` it intially assess `a == 0`. If this is false, the `&&` resolves to false regardless of what the right-hand operand is. Hence the right hand doesn't get executed. If `a== 0` is true, then Julia executes the right hand operand and `return "Error - ..."` is run, breaking us away from the function, and returning control to the parent function/Julia/OS/etc.

9. Instead of returning control to the parent function/Julia/OS/etc., it crashes the program and spits out the DomainError and explanation, so that the user can debug and fix their code. DomainError basically means (in mathematical language) "input is not an element of the function's domain"

## 2(b) Let's try some examples



In [12]:
function print_quadratic_roots(coefficients::Vector{<:Number}, roots::Vector{<:Number})
    print("The equation \
    $(abs(coefficients[1]) == 1 ? (coefficients[1] < 0 ? "-" : "") : coefficients[1])x² \
    $(coefficients[2] < 0 ? "- " : "+ ")$(abs(coefficients[2]))x \
    $(coefficients[3] < 0 ? "- " : "+ ")$(abs(coefficients[3])) = 0 ")
    if length(roots) == 0
        println("has no real roots.")
    elseif length(roots) == 1
        println("has a single (real) root $(roots[1])")
    else
        println("has the real roots $(roots[1]) and $(roots[2])")
    end
    return Nothing
end

examples = [[1,-5,6], [1,2,3], [1,7,0], [3,-3π,-6π^2], [1,-4,4], [-1,-1,1]]

for example in examples
    roots = real_roots_of_quadratic(example[1], example[2], example[3])
    print_quadratic_roots(example, roots)
end

The equation x² - 5.0x + 6.0 = 0 has the real roots 3.0 and 2.0
The equation x² + 2.0x + 3.0 = 0 has no real roots.
The equation x² + 7.0x + 0.0 = 0 has the real roots 0.0 and -7.0
The equation 3.0x² - 9.42477796076938x - 59.21762640653615 = 0 has the real roots 6.283185307179586 and -3.141592653589793
The equation x² - 4.0x + 4.0 = 0 has a single (real) root 2.0
The equation -x² - 1.0x + 1.0 = 0 has the real roots -1.618033988749895 and 0.6180339887498949


1. I added 3x^2 - 3pi*x - 6pi^2 and x^2 - 4x + 4. It works as expected.

2. I've added the function as requested! The `::Vector{<:Number}` type annotation tells Julia to accept inputs which are vectors of _any_ type of number (`Int`, `Float`, `BigInt`, etc.)

3. Ok, i think I went a little beyond the scope of the questions, but it does achieve the desired effect of the question. Using terney logic operators, i've made it that `-1x^2 + -2x + -3` prints as `-x^2 - 2x - 3` which is significnatly pretier. I've used some newlines in the print statement to make it a little more readable.

## 2(c) Creating a testing framework for our quadratic solver

In [19]:
using Random
"""
This function generates `num_tests` random triples of coefficients and checks that the function `real_roots_of_quadratic()` does its job. The return value is `true` if the test passed, otherwise it is `false`.
"""
function test_real_roots_of_quadratic(;num_tests=10000, seed=42, coeff_min=-1000., coeff_max=1000.)
    Random.seed!(seed)
    test_passed = true
    for _ in 1:num_tests
        a, b, c = (coeff_max - coeff_min)rand(3) .+ coeff_min # Uniform values in range [-1000, 1000]
        roots = real_roots_of_quadratic(a,b,c)
        for x in roots
            err = a*x^2 + b*x + c
            test_passed = (test_passed && isapprox(err, 0.0, atol = 1e-8))
        end
    end
    return test_passed
end

function test_random_roots_of_quadratic(;num_tests=10000, seed=42, root_min=-1000., root_max=1000.)
    Random.seed!(seed)
    test_passed = true
    for _ in 1:num_tests
        coes::Array{Float64} = []
        # We will ignore the case of 0 real roots, because
        # there is nothing to test in that specific situation
        # Hence, lets generate an integer, 1 or 2:
        root_count = Int(floor(2rand()+1))
        roots = (root_max - root_min)rand(root_count) .+ (root_min)
        sort!(roots)
        if root_count == 2
           coes = [1, -roots[2] - roots[1], roots[2]*roots[1]]  # a = roots[1]; b = roots[2]; (x-a)(x-b) = x² - ax - bx + ab
        elseif root_count == 1
            coes = [1, -2roots[1], roots[1]*roots[1]]  # a = roots[1]; (x-a)(x-a) = x² - 2ax + a²
        end
        calced_roots = real_roots_of_quadratic(coes[1],coes[2],coes[3])
        sort!(calced_roots)
        if length(roots) != length(calced_roots)
            test_passed = false
        else
            for (a,b) in zip(roots, calced_roots)
                test_passed = (test_passed && isapprox(a - b, 0.0, atol = 1e-8))
            end
        end 
    end
    return test_passed
end

test_real_roots_of_quadratic() ? println("Test passed") : println("Test failed!")
test_random_roots_of_quadratic() ? println("Test passed") : println("Test failed!")

Test passed
Test passed


1. `rand(3)` creates a vector of 3 uniform indenpendently and identically destributed floats, between 0 and 1. `2000rand(3)` then multiplies those 3 elements by 2000, effectively giving us numbers in the range [0, 2000]. Finally, `.- 1000.` performs an element-wise subtraction of 1000.0 (a float) on the vector, giving us the desired range [-1000, 1000]. Finally, to achieve the range [-5, 5] I would rewrite the expression as `a, b, c = 10rand(3) .- 5.`. 

2. i did this. Initally I had `(max - min)rand(3) .- min` and it was compling properly and passing the test, but when i investigated the numbers it was producing it was not in bounds! I eventually realised that the line should be `(max - min)rand(3) .+ min`!

3. If there are no real roots, `roots == []`, so when we come to `for x in roots` nothing happens, because there is no x in roots. So, no (0) itterations (**** CHECK)

4. I basically described this in 2.! If I placed `.- coeff_min` in the line where we generate the coefficents, the code would complie, run, and pass all tests, so the actually 'bug' is quite subtle. Only in investigating the code more closely was I able to confirm its exisitence.

5. For `root_count` i tried to find an equivilent to Python's `random.randint()` but couldn't find anything like that in the Julia docs, so i had to generate the numbers in the range i wanted manually. my code randomly picks a root count, we skipp 0 because if the root count is 0, there are no real roots to check we're calculating correctly. Next we generate a vector with root_count number of roots in some range (the default is [-1000,1000]). Next, we calculate the coefficents. This is just classic year 8 level algebra. The calculation is dependent on root count though. Finally, we ship those calculated coefficents to the `real_roots_of_quadratic` function, and then I sort the output (that way I can be sure that the correct roots are being compared). Finally, i iterate over the roots, zipped with the calculated roots, and I check that the difference between each pair is approximately equal to 0, down to some error term. If they are, test_passed remains true, else, it is set to false. After 10,000 itterations (by default), we return `test_passed`.

## 2(d) Expanding our quadratic solver to $\mathbb{C}$

In [30]:
function roots_of_quadratic(a::Number, b::Number, c::Number)
    roots::Array{ComplexF64} = []
    Δ = b^2 - 4a*c
    if Δ == 0
        roots = [-b/(2a)]
    else
        Δ = Δ < 0 ? Complex(Δ) : Δ
        roots = [-b + √Δ, -b - √Δ] / (2a)
    end
    return roots
end

@show roots_of_quadratic(1,2,3)
@show roots_of_quadratic(-1,2,3)

roots_of_quadratic(1, 2, 3) = ComplexF64[-1.0 + 1.4142135623730951im, -1.0 - 1.4142135623730951im]
roots_of_quadratic(-1, 2, 3) = ComplexF64[-1.0 + 0.0im, 3.0 + 0.0im]


2-element Vector{ComplexF64}:
 -1.0 + 0.0im
  3.0 + 0.0im

1. I removed the redunant case and added a ternery for the case that the quadratic has 2 roots (regardless of whether they're real or complex). In the case they are complex, the ternery dynamically converts the Delta to a complex number, and otherwise leaves it as a real number.